# 🎨 AI Fashion Stylist — RAG + Gemini 2.0 Flash

## 📌 Main Idea

This project uses Generative AI and Retrieval-Augmented Generation (RAG) techniques to build an AI-powered fashion recommendation system — a stylist agent that understands semantic meaning from product descriptions and images, offering outfit suggestions contextually using AI.

We generate text embeddings from product metadata (name, category, color, etc.) using sentence-transformers. Then, we use cosine similarity-based vector search to find the most relevant products for a given user query.

Next, we pass these retrieved product details to Gemini 2.0 Flash, which acts as a stylist, providing contextual outfit recommendations, styling suggestions, and more.

## 🧠 Embeddings & Retrieval
- Product descriptions are created by combining:

    - productDisplayName
    - masterCategory
    - subCategory
    - articleType
    - baseColour
    - usage

- Embeddings are generated using all-MiniLM-L6-v2 model from sentence-transformers.

- Cosine similarity is computed between the user query embedding and product embeddings.

- Top 5 matching products are retrieved.

## 👗 Fashion AI Agent
- Uses Gemini 2.0 Flash API.

- Receives top 5 product details from vector search.

- Provides:

   - Best matching product

   - Reasoning for the match

   - Styling advice (accessories, shoes, bags)

   - Occasion and color coordination tips

## 📑 Notebook Features
- ✅ Merged product dataset with image filenames

- ✅ Embedded product descriptions using sentence-transformers

- ✅ Cosine similarity-based vector search retrieval

- ✅ Gemini 2.0 Flash AI stylist integration

- ✅ Image preview function for local or hosted images

- ✅ Seamlessly runs on Google Colab or Kaggle

## ✅ Progress and Checklist
 - Prepare dataset merged with image filenames and selected columns

 - Embed name + description with sentence-transformers

 - Add product metadata to vectors

 - Retrieve top-k results with cosine similarity

 - AI stylist agent using Gemini 2.0 Flash

 - Image preview integration

 - (Optional) Multi-modal image embeddings (CLIP / NomicAI)

 - (Optional) Streamlit/Gradio frontend

 - (Optional) Write a blog post 📖

In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from IPython.display import Image, display, Markdown

2025-04-20 18:32:20.570498: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745173940.803471      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745173940.868590      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
images_df = pd.read_csv('/kaggle/input/fashion-product-images-dataset/fashion-dataset/images.csv')
styles_df = pd.read_csv('/kaggle/input/fashion-product-images-dataset/fashion-dataset/styles.csv', on_bad_lines='skip')

images_df['id'] = images_df['filename'].str.split('.').str[0]
images_df['id'] = images_df['id'].astype('int')
merged_df = styles_df.merge(images_df, left_on = 'id', right_on='id', how='left')

In [5]:
merged_df['description'] = merged_df.apply(
    lambda x: f"{x['productDisplayName']}, {x['masterCategory']}, {x['subCategory']}, {x['articleType']}, {x['baseColour']}, {x['usage']}, {x['gender']}, {x['season']}", axis=1
)

In [6]:
merged_df.head()

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,filename,link,description
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011.0,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg,http://assets.myntassets.com/v1/images/style/p...,"Turtle Check Men Navy Blue Shirt, Apparel, Top..."
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012.0,Casual,Peter England Men Party Blue Jeans,39386.jpg,http://assets.myntassets.com/v1/images/style/p...,"Peter England Men Party Blue Jeans, Apparel, B..."
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016.0,Casual,Titan Women Silver Watch,59263.jpg,http://assets.myntassets.com/v1/images/style/p...,"Titan Women Silver Watch, Accessories, Watches..."
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011.0,Casual,Manchester United Men Solid Black Track Pants,21379.jpg,http://assets.myntassets.com/v1/images/style/p...,"Manchester United Men Solid Black Track Pants,..."
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012.0,Casual,Puma Men Grey T-shirt,53759.jpg,http://assets.myntassets.com/v1/images/style/p...,"Puma Men Grey T-shirt, Apparel, Topwear, Tshir..."


In [7]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(merged_df['description'].tolist())

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1389 [00:00<?, ?it/s]

In [8]:
def cosine_similarity(a, b):
    a_norm = np.linalg.norm(a)
    b_norm = np.linalg.norm(b)
    return np.dot(a, b) / (a_norm * b_norm)

def retrieve_products(user_query, top_k=5):
    query_embedding = embedder.encode([user_query])[0]
    similarities = np.array([cosine_similarity(query_embedding, emb) for emb in embeddings])
    top_indices = np.argsort(similarities)[::-1][:top_k]
    return merged_df.iloc[top_indices]

In [9]:
def show_product_images(df, local_folder_path=None, hosted_url_prefix=None):
    for _, row in df.iterrows():
        if local_folder_path:
            image_path = f"{local_folder_path}/{row['filename']}.jpg"
            display(Image(filename=image_path))
        elif hosted_url_prefix:
            image_url = f"{row['link']}"
            print(row[['productDisplayName', 'masterCategory', 'subCategory', 'articleType', 'baseColour']])
            display(Image(url=image_url))

In [10]:
from google import genai
from google.genai import types

from google.api_core import retry
from google.colab import userdata

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)
client = genai.Client(api_key=GOOGLE_API_KEY)

In [11]:
def style_ai_agent(user_query, retrieved_products):
    product_summaries = []
    for _, row in retrieved_products.iterrows():
        summary = (
            f"Product: {row['productDisplayName']}\n"
            f"Category: {row['masterCategory']} > {row['subCategory']} > {row['articleType']}\n"
            f"Color: {row['baseColour']}\n"
            f"Usage: {row['usage']}\n"
        )
        product_summaries.append(summary)

    combined_summaries = "\n".join(product_summaries)

    prompt = (
        f"You are a professional fashion stylist AI agent.\n\n"
        f"A user is looking for: \"{user_query}\"\n\n"
        f"Here are 5 relevant products retrieved from the fashion catalog:\n"
        f"{combined_summaries}\n\n"
        "Based on these:\n"
        "- Suggest which product would best match the user's query.\n"
        "- Explain your reasoning.\n"
        "- Recommend how to style this outfit (accessories, shoes, bag etc.)\n"
        "- Optionally suggest color coordination and occasion-fit tips."
    )
    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)
 
    return response.text

In [12]:
user_query = "vibrant dress for brunch"
retrieved_products = retrieve_products(user_query, top_k=5)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
show_product_images(retrieved_products, hosted_url_prefix=" ")

productDisplayName    Forever New Women's Waterfall Sleeve Shift Bla...
masterCategory                                                  Apparel
subCategory                                                       Dress
articleType                                                     Dresses
baseColour                                                        Black
Name: 19261, dtype: object


productDisplayName    Mineral Blue & Green Printed Dress
masterCategory                                   Apparel
subCategory                                        Dress
articleType                                      Dresses
baseColour                                          Blue
Name: 21139, dtype: object


productDisplayName    Bwitch Turquiose Blue Nightdress
masterCategory                                 Apparel
subCategory                   Loungewear and Nightwear
articleType                                 Nightdress
baseColour                              Turquoise Blue
Name: 13624, dtype: object


productDisplayName    French Connection Teal & Brown Printed Dress
masterCategory                                             Apparel
subCategory                                                  Dress
articleType                                                Dresses
baseColour                                                    Teal
Name: 29847, dtype: object


productDisplayName    Forever New Women Blossom Silk Cream Dress
masterCategory                                           Apparel
subCategory                                                Dress
articleType                                              Dresses
baseColour                                                 Cream
Name: 12511, dtype: object


In [14]:
advice = style_ai_agent(user_query, retrieved_products)
Markdown(advice)

Okay, based on the user's query "vibrant dress for brunch" and the provided product catalog, the **Mineral Blue & Green Printed Dress** is the best match.

**Reasoning:**

*   **Vibrant:** The description includes "Blue & Green Printed," suggesting a colorful and lively design, which aligns with the "vibrant" requirement.
*   **Dress:** It is explicitly categorized as a dress.
*   **Brunch Appropriateness:** The "Casual" usage tag indicates it's suitable for a relaxed occasion like brunch.
*   **Other Options:** The other dresses are less suitable:
    *   The black and cream dresses are neutral and lack vibrancy.
    *   The Turquoise Blue Nightdress, while vibrant in color, is categorized as nightwear and not appropriate for brunch.
    *   The Teal & Brown Printed Dress might be colorful, but teal and brown can be more muted than vibrant, and the color combination might not be as brunch-appropriate.

**Styling Recommendation for the Mineral Blue & Green Printed Dress:**

To create a complete and stylish brunch outfit, here's how I recommend styling the dress:

*   **Shoes:**
    *   **Option 1 (Casual & Comfortable):** White sneakers or canvas shoes. This adds a relaxed and trendy vibe, perfect for a casual brunch.
    *   **Option 2 (Slightly Dressier):** Nude or white espadrille wedges or sandals. These elevate the look a bit while still maintaining comfort.
    *   **Color Coordination:** If the print has hints of yellow or orange, sandals in those colors would also work well.
*   **Bag:**
    *   **Option 1 (Crossbody/Shoulder Bag):** A small to medium-sized crossbody bag or shoulder bag in a neutral color like beige, white, or a light brown. This is practical and keeps your hands free.
    *   **Option 2 (Straw/Woven Bag):** A straw tote bag or a woven bag would complement the casual brunch vibe perfectly.
*   **Accessories:**
    *   **Jewelry:** Keep it simple. A delicate gold or silver necklace, small hoop earrings, or a simple bracelet would be ideal. Avoid anything too flashy.
    *   **Sunglasses:** Stylish sunglasses are a must for a sunny brunch. Choose a shape that complements your face.
    *   **Hat (Optional):** A wide-brimmed straw hat can add a touch of elegance and protect you from the sun if you're dining outdoors.
    *   **Light Layer (If Needed):** If the weather is unpredictable, bring a light denim jacket, a white cardigan, or a linen blazer.
*   **Hair & Makeup:**
    *   **Hair:** Style your hair in a relaxed way – loose waves, a messy bun, or a simple ponytail.
    *   **Makeup:** Opt for a natural and fresh makeup look. A tinted moisturizer, light blush, mascara, and a tinted lip balm or gloss will suffice.

**Color Coordination and Occasion-Fit Tips:**

*   **Color Coordination:** Focus on complementing the blue and green in the dress print. Nude, white, beige, and light browns are safe and versatile choices. If you want to add a pop of color, consider accessories in coral, yellow, or a lighter shade of green or blue, depending on the specific shades in the print.
*   **Occasion-Fit:** Brunch is generally a relaxed and daytime affair. Avoid anything too formal or overtly sexy. The key is to look stylish and put-together while remaining comfortable. The Mineral Blue & Green Printed Dress, when styled as suggested, perfectly balances these elements.
